<a href="https://colab.research.google.com/github/MarioViniciusBC/NumPy/blob/main/NumPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NumPy

In [ ]:
import numpy as np

## Broadcasting

O broadcasting é simplesmente um conjunto de regras para aplicar ufuncs binárias (por exemplo, adição, subtração, multiplicação, etc.) em arrays de tamanhos diferentes.

In [ ]:
list_a = [1, 2, 3, 4]

list_b = []
for i in list_a:
  list_b.append(i + 5)

print(list_b)

[6, 7, 8, 9]


O Python nativo não sabe como somar um número inteiro (como o **5**) diretamente a uma coleção de itens (a lista). Para fazer isso, somos obrigados a criar uma lista auxiliar vazia e usar um laço de repetição (`for`) para iterar por cada item, somar o valor e anexar (`append`) o resultado um por um. É um processo mais longo e verboso.

In [ ]:
array_a = np.array(list_a)
array_a + 5

array([6, 7, 8, 9])

o converter a lista para um `np.array`, o comportamento muda. Quando executamos `array_a + 5`, o NumPy entende que a intenção é aplicar essa operação matemática a todos os elementos da matriz simultaneamente, de forma automática.

## Regras

### **Regra 1: Preenchimento com "1" à esquerda (Alinhamento)**

*Se os dois arrays têm um número diferente de dimensões, o NumPy adiciona "1s" à esquerda do shape do array menor até que ambos tenham a mesma quantidade de dimensões.*

**Na prática:** Imagine que você tem uma *matriz 2D* e um *array 1D*.

* `M` tem shape `(2, 3)` (2 linhas, 3 colunas)

* `a` tem shape `(3,)` (apenas 3 itens)

**O que o NumPy faz:** Ele não consegue comparar `(2, 3)` com `(3,)`. Então, ele aplica a **Regra 1** e adiciona um "**1**" na frente do array menor.

O shape de `a` passa a ser virtualmente `(1, 3)`.

In [ ]:
M = np.ones((2, 3))
a = np.arange(3)

print(M.shape)
print(a.shape)

(2, 3)
(3,)


Vemos pela **regra 1** que o array `a` tem menos dimensões, então o preenchemos à esquerda com uns:

* `M.shape` -> (2, 3)

* `a.shape` -> (1, 3)

As formas coincidem, e vemos que a forma final será (2, 3):

In [ ]:
M + a

array([[1., 2., 3.],
       [1., 2., 3.]])

### **Regra 2: O Esticamento (Stretch)**

*Se os arrays têm a mesma quantidade de dimensões, mas os tamanhos são diferentes em alguma delas, o NumPy procura por dimensões de tamanho "1" e as "estica" para igualar ao tamanho do outro array*.

**Na prática (continuando o exemplo anterior):**

* `M` tem shape `(2, 3)`

* `a` agora tem shape `(1, 3)` (graças à Regra 1)

**O que o NumPy faz:** Ele compara as dimensões. As colunas já são iguais (**3** e **3**). Mas as linhas são diferentes (**2** e **1**). Como a regra diz que o **1** pode ser esticado, o NumPy *"copia"* aquela única linha do array `a` para que ele fique com shape `(2, 3)`. Agora a conta pode ser feita!

A seguir, vemos um segundo exemplo:

In [ ]:
# 1. Criando a matriz Mat com shape (2, 3)
M = np.array([[1, 2, 3],
                [4, 5, 6]])

# 2. Criando o array 'a' com shape (1, 3)
a = np.array([[10, 20, 30]])

print(f"Shape de M: {M.shape}")
print(f"Shape de a: {a.shape}\n")

# 3. A mágica da Regra 2 acontece aqui:
resultado = M + a

print("Resultado de M + a:")
print(resultado)

Shape de M: (2, 3)
Shape de a: (1, 3)

Resultado de M + a:
[[11 22 33]
 [14 25 36]]


Como os dois arrays já têm **2** dimensões (**Regra 1** não precisa agir aqui), o NumPy pula direto para a **Regra 2** e compara os tamanhos (shapes):

1. **Colunas:** `M` tem **3** colunas, `a` tem **3** colunas. Perfeito, combinam!

2. **Linhas:** `M` tem **2** linhas, `a` tem **1** linha.

É aqui que a **Regra 2** entra em ação. O NumPy vê esse **1** e diz: "Opa, posso esticar isso aqui!". Ele cria uma cópia virtual dessa única linha de `a` para que ela vire uma matriz de **2** linhas, ficando com o shape `(2, 3)` igualzinho ao `M`.

~~~python
# 'a' original (1, 3)
[[10, 20, 30]]

# 'a' esticado virtualmente pela Regra 2 para (2, 3)
[[10, 20, 30],
 [10, 20, 30]]
~~~

Depois disso, ele simplesmente soma `M` com essa nova versão esticada de `a`, elemento por elemento:

* **Linha 1:** `[1, 2, 3] + [10, 20, 30] = [11, 22, 33]`

* **Linha 2:** `[4, 5, 6] + [10, 20, 30] = [14, 25, 36]`

### **Regra 3: A Parede de Tijolos (O Erro)**

*Se após aplicar as regras **1** e **2**, os tamanhos das dimensões ainda não baterem e nenhum deles for "**1**", o NumPy joga a toalha e devolve um erro.*

**Na prática:** Imagine tentar somar uma matriz `(3, 2)` com um array de shape `(3,)`.

* **Passo 1:** `(3, 2)` e `(3,)` -> Pela **Regra 1**, o menor vira `(1, 3)`.

* **Passo 2:** O NumPy compara `(3, 2)` com `(1, 3)`.

* **Passo 3:** O **1** na primeira dimensão pode virar **3** (**Regra 2**). Eles ficariam `(3, 2)` e `(3, 3)`.

**O Problema:** Na segunda dimensão, temos um 2 e um 3. Eles são diferentes, e **nenhum deles é 1**. O NumPy não sabe o que fazer. Ele não pode esticar um array de **2** colunas para virar **3** colunas magicamente.

**Resultado:** `ValueError: operands could not be broadcast together with shapes (3,2) (3,).`

In [ ]:
import numpy as np

# 1. Criando a matriz M com shape (3, 2) - 3 linhas, 2 colunas
M = np.ones((3, 2))

# 2. Criando o array 'a' com shape (3,) - 3 elementos
a = np.array([10, 20, 30])

print(f"Shape de M: {M.shape}")
print(f"Shape de a: {a.shape}\n")

# 3. A parede de tijolos (Regra 3) acontece aqui:
print("Tentando calcular M + a...")
resultado = M + a
# "operands could not be broadcast together with shapes (3,2) (3,)"

### Dimensão e Tamanho

Para o NumPy, precisamos separar dois conceitos que parecem iguais, mas são diferentes: o **Número de Dimensões** e o **Tamanho da Dimensão** (Shape).

Vamos separar os dois:

1. O que é uma "Dimensão" (ou Eixo / Axis)
Pense na dimensão como a direção geométrica para onde os seus dados podem crescer.

* **Dimensão (1D):** É uma reta. Pode ser uma linha ou uma coluna isolada. Só vai para uma direção. Ex: um vetor [1, 2, 3].

* **Dimensões (2D):** É uma superfície, uma grade. Ela tem duas direções: para baixo (que chamamos de linhas) e para o lado (que chamamos de colunas). É uma matriz plana.

* **Dimensões (3D):** É um bloco ou cubo. Tem linhas, colunas e "profundidade".

Portanto, em um array 2D, a "linha" é apenas a primeira dimensão, e a "coluna" é a segunda dimensão.

2. O que é o "Tamanho" (Shape)
O tamanho (shape) é a quantidade de itens que existe dentro de cada uma dessas dimensões.

Por exemplo, se uma matriz tem o shape (4, 3), isso significa que:

Ela tem 2 dimensões (porque tem dois números dentro do parênteses).

* Na dimensão 1 (linhas), o tamanho é 4.

* Na dimensão 2 (colunas), o tamanho é 3.